In [1]:
import requests
import json

import pytezos as ptz
from pytezos import Key

import demo.demo as demo

We're using https://packages.ligolang.org/contract/Permit-Cameligo which itself extends `ligo-extendable-fa2` to add a Permit implementation.

In [2]:
!ligo compile contract permit-cameligo/src/main.mligo > permit-contract.tz

In [3]:
!ligo compile contract demo/staking-contract.mligo > staking-contract.tz

Make sure the API is running by visiting http://127.0.0.1:8000/docs. The API is written in Python using FastAPI and can be started with `uvicorn src.main:app --reload`.

# Deploying the contracts

We're using a local network for this demo (typically using Flextesa), but this has been tested on Ghostnet as well. On Ghostnet, operations and balance changes seem to be correctly picked by the indexers, even for a non-revealed account.

In [4]:
TEZOS_RPC = "http://localhost:20000"

In [5]:
admin_key = Key.from_encoded_key("edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq")
admin_key.public_key_hash()
admin = ptz.pytezos.using(TEZOS_RPC, admin_key)
admin.balance()

Decimal('1999996.211299')

`admin` is a PyTezosClient object. It can query Tezos RPCs, inspect contracts' storage and send operations. This key is used in the API as well, and this is the one doing all the requests for the accounts we're going to define.

Its address is `tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb` and is also known as “alice” among Tezos developers.

In [6]:
admin.key.public_key_hash()

'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb'

In [7]:
nft_contract = demo.deploy_permit(admin)
nft_contract.address

'KT18mCT5nhvSfidacdU9rAL9KqEvDWQaqJys'

In [8]:
staking_contract = demo.deploy_staking_contract(admin, nft_contract)
staking_contract.address

'KT1NYUCNsd5NybHVr3UVYZE4WQmWQn4VJddi'

# Minting NFTs for three new accounts

The `demo/demo.py` file contains all the code necessary for the demo: generation of off-chain permits, of the hashes and the signatures, and communication with the API.

In [9]:
demo = demo.Demo(nft_contract, staking_contract)

We generate 3 test accounts. None of them own any tez, and they cannot post transactions on Tezos themselves.

In [10]:
keys = [Key.generate() for i in range(3)]
senders = [k.public_key_hash() for k in keys]
senders

['tz1fYpfLTfErLMvWEtgAcASvcgNnkgAhcHme',
 'tz1dYe5EhqCULrB2d4ASBpibFB7gA6xUqUi5',
 'tz1MxoG2c9Zzrvap4BriZ3qdpKbrm2pxBTJG']

We call the API to request it to mint 100 tokens to all these accounts. These calls are done in parallel, but the API automatically groups them in a single transaction.

In [11]:
_ = demo.mint_requests(senders)

calling http://127.0.0.1:8000/operation with sender=tz1fYpfLTfErLMvWEtgAcASvcgNnkgAhcHme
calling http://127.0.0.1:8000/operation with sender=tz1dYe5EhqCULrB2d4ASBpibFB7gA6xUqUi5
calling http://127.0.0.1:8000/operation with sender=tz1MxoG2c9Zzrvap4BriZ3qdpKbrm2pxBTJG


We can inspect the NFT ledger to see that all the addresses received tokens.

In [12]:
for sender in senders:
    minted = nft_contract.storage["ledger"][(sender, 0)]()
    print(f"{sender} owns {minted} tokens.")

tz1fYpfLTfErLMvWEtgAcASvcgNnkgAhcHme owns 100 tokens.
tz1dYe5EhqCULrB2d4ASBpibFB7gA6xUqUi5 owns 100 tokens.
tz1MxoG2c9Zzrvap4BriZ3qdpKbrm2pxBTJG owns 100 tokens.
<Response [200]>
<Response [200]>
<Response [200]>


# Transferring the NFTs by signing off-chain permits

In this section, we're going to send 10 tokens to the “staking” contract. This contract does nothing special except from receiving the 10 tokens and saving the staker's address in its own storage. 

To be more precise, when we call `stake(10, address)`, the staking contract emits a new `transfer` operation, which is the one for which we signed the permit. If this permit wasn't signed, the whole transaction would fail.

In [13]:
staking_contract.stake


Properties
.key		tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb
.shell		['http://localhost:20000']
.address	KT1NYUCNsd5NybHVr3UVYZE4WQmWQn4VJddi
.block_id	head
.entrypoint	stake

Builtin
(*args, **kwargs)	# build transaction parameters (see typedef)

Typedef
$stake:
	( nat, address )

$nat:
	int  /* Natural number */

$address:
	str  /* Base58 encoded `tz` or `KT` address */


Helpers
.decode()
.encode()

We can now sign the permits off-chain. This is the most complicated part (hidden inside the `Demo` class), as it involves hashing a few structures related to the contracts' entrypoints, and signing them. We're in the process of building SDKs for this.

The hashes are then signed by each key (off-chain), and the permits can be posted on-chain by any account. They are then stored in the NFT contract.

In [14]:
_ = demo.permit_requests(keys)

calling http://127.0.0.1:8000/operation with sender=tz1fYpfLTfErLMvWEtgAcASvcgNnkgAhcHme
calling http://127.0.0.1:8000/operation with sender=tz1dYe5EhqCULrB2d4ASBpibFB7gA6xUqUi5
calling http://127.0.0.1:8000/operation with sender=tz1MxoG2c9Zzrvap4BriZ3qdpKbrm2pxBTJG
<Response [200]>
<Response [409]>
<Response [409]>


However, note that this demo uses a contract limited to a single, global counter; for technical reasons, **only one of these permits is valid** and the two others will be refused by the API.

In production, we recommend using a more flexible smart contract. We leave this one in the demo to show that the API is somewhat robust to invalid transactions.

Finally, we can stake the 10 tokens. Again, this call could be posted by any account, not necessarily the API. As we don't know which user successfully posted a permit, we try all three.

In [17]:
for key in keys:
    demo.stake_request(key.public_key_hash())

But we can check that the tokens have indeed been transfered to the staking contract:

In [18]:
nft_contract.storage["ledger"][(staking_contract.address, 0)]()

10